In [ ]:
from ultralytics import YOLO
import os
from lxml import etree
from sklearn.model_selection import train_test_split
from pathlib import Path
import cv2

In [ ]:
xml_files_path = '../dataset/annotations/'
image_files_path = '../dataset/images/'
labels_dict = {'filepath': [], 'xcenter': [], 'ycenter': [], 'width': [], 'height': []}

for xml_filename in os.listdir(xml_files_path):
    tree = etree.parse(f"{xml_files_path}/{xml_filename}")
    for dim in tree.xpath("size"):
        img_width = int(dim.xpath("width")[0].text)
        img_height = int(dim.xpath("height")[0].text)
    for dim in tree.xpath("object/bndbox"):
        xmin = int(dim.xpath("xmin")[0].text)
        ymin = int(dim.xpath("ymin")[0].text)
        xmax = int(dim.xpath("xmax")[0].text)
        ymax = int(dim.xpath("ymax")[0].text)

    x_center = (xmin + xmax) / 2 / img_width
    y_center = (ymin + ymax) / 2 / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height

    image_filename = xml_filename.replace('annotations', 'images').replace('.xml', '.png')
    labels_dict['filepath'].append(image_filename)
    labels_dict['xcenter'].append(x_center)
    labels_dict['ycenter'].append(y_center)
    labels_dict['width'].append(width)
    labels_dict['height'].append(height)

In [ ]:
image_paths = labels_dict['filepath']
labels = list(zip(labels_dict['xcenter'], labels_dict['ycenter'],
                  labels_dict['width'], labels_dict['height']))

train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.12, random_state=42)

In [ ]:
output_dir = '../yolo_dataset'
os.makedirs(f'{output_dir}/images/train', exist_ok=True)
os.makedirs(f'{output_dir}/images/val', exist_ok=True)
os.makedirs(f'{output_dir}/labels/train', exist_ok=True)
os.makedirs(f'{output_dir}/labels/val', exist_ok=True)

for i, (image_path, image_label) in enumerate(zip(train_image_paths, train_labels)):
    img = cv2.imread(f"{image_files_path}/{image_path}")
    label_path = Path(image_path).stem + '.txt'

    cv2.imwrite(f'{output_dir}/images/train/{Path(image_path).name}', img)
    with open(f'{output_dir}/labels/train/{label_path}', 'w') as f:
        f.write(f'0 {image_label[0]} {image_label[1]} {image_label[2]} {image_label[3]}\n')

for i, (image_path, image_label) in enumerate(zip(test_image_paths, test_labels)):
    img = cv2.imread(f"{image_files_path}/{image_path}")
    label_path = Path(image_path).stem + '.txt'

    cv2.imwrite(f'{output_dir}/images/val/{Path(image_path).name}', img)
    with open(f'{output_dir}/labels/val/{label_path}', 'w') as f:
        f.write(f'0 {image_label[0]} {image_label[1]} {image_label[2]} {image_label[3]}\n')

In [ ]:
train_images_path = '../yolo_dataset/images/train'
val_images_path = '../yolo_dataset/images/val'
classes = ['license_plate']

yaml_content = f"""
train: {os.path.abspath(train_images_path)}
val: {os.path.abspath(val_images_path)}

nc: {len(classes)}
names: {classes}
"""

yaml_file_path = 'license_plate.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

In [ ]:
# Load the model
model = YOLO("../src/models/yolo11n.pt")

In [ ]:
# Train the model
model.train(
    data='license_plate.yaml',
    epochs=10,
    batch=16,
    imgsz=640,
    workers=4
)